<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/neural_networks/RNN/review_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rekurencyjna sieć neuronowa - Klasyfikacja recezji filmowych

### Import bibliotek

In [1]:
%tensorflow_version 2.x
import numpy as np
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

TensorFlow 2.x selected.


### Załadowanie danych

In [2]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
!unzip -q reviews.zip

--2019-12-29 09:19:08--  https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.184.128, 2a00:1450:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.184.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42878657 (41M) [application/x-zip-compressed]
Saving to: ‘reviews.zip’

reviews.zip         100%[===================>]  40.89M   139MB/s    in 0.3s    

2019-12-29 09:19:09 (139 MB/s) - ‘reviews.zip’ saved [42878657/42878657]



### Utworzenie katalogów

In [0]:
data_dir = './reviews'
train_dir = os.path.join(data_dir, 'train')

train_texts = []
train_labels = []

for label_type in ['neg', 'pos']:
  dir_name = os.path.join(train_dir, label_type)
  for fname in os.listdir(dir_name):
    if fname[-4:] == '.txt':
      f = open(os.path.join(dir_name, fname))
      train_texts.append(f.read())
      f.close()
      if label_type == 'neg':
        train_labels.append(0)
      else:
        train_labels.append(1)

In [0]:
test_dir = os.path.join(data_dir, 'test')

test_texts = []
test_labels = []

for label_type in ['neg', 'pos']:
  dir_name = os.path.join(test_dir, label_type)
  for fname in os.listdir(dir_name):
    if fname[-4:] == '.txt':
      f = open(os.path.join(dir_name, fname))
      test_texts.append(f.read())
      f.close()
      if label_type == 'neg':
        test_labels.append(0)
      else:
        test_labels.append(1)

### Esploracja danych

In [5]:
train_texts[:10]

["This is actually a pretty bad film. The ideology is not as perverse as in those films Collins made later. However, my main misgivings about the film are that it is implausible and quite frankly boring for a long time. The whole concept of an ex-SAS man joining terrorists for no particular reason isn't very convincing and you can't help wondering why a group of highly organized terrorists (who later become pretty clueless) fall for it. The film starts with a pretty powerful scene but then meanders for quite a long time building up towards the great finale. Overall, I think Who dares wins could have been an interesting 45 minutes episode of The Professionals but the story doesn't carry a feature film. Although reasonably successful at the time this film initiated the demise of Collins' career who in the eighties mainly made cheap and dubious soldier-of-fortune or army films. Pity, because he actually is quite a versatile actor but at the end of the day Martin Shaw chose his roles more 

In [6]:
train_labels[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [7]:
train_labels[-10:]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [0]:
maxlen = 100         # skracamy recenzje do 100 słów
num_words = 10000    # 10000 najczęściej pojawiających się słów
embedding_dim = 100

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_texts)

In [9]:
list(tokenizer.index_word.items())[:20]

[(1, 'the'),
 (2, 'and'),
 (3, 'a'),
 (4, 'of'),
 (5, 'to'),
 (6, 'is'),
 (7, 'br'),
 (8, 'in'),
 (9, 'it'),
 (10, 'i'),
 (11, 'this'),
 (12, 'that'),
 (13, 'was'),
 (14, 'as'),
 (15, 'for'),
 (16, 'with'),
 (17, 'movie'),
 (18, 'but'),
 (19, 'film'),
 (20, 'on')]

In [10]:
sequences = tokenizer.texts_to_sequences(train_texts)
print(sequences[:3])

[[11, 6, 162, 3, 181, 75, 19, 1, 9654, 6, 21, 14, 8288, 14, 8, 145, 105, 7364, 90, 300, 187, 58, 290, 41, 1, 19, 23, 12, 9, 6, 4010, 2, 176, 2032, 354, 15, 3, 193, 55, 1, 223, 1117, 4, 32, 1230, 129, 7916, 4844, 15, 54, 840, 279, 215, 52, 1075, 2, 22, 188, 335, 1525, 135, 3, 601, 4, 542, 5844, 4844, 34, 300, 410, 181, 5745, 804, 15, 9, 1, 19, 514, 16, 3, 181, 972, 133, 18, 92, 15, 176, 3, 193, 55, 1425, 53, 946, 1, 84, 1957, 441, 10, 101, 34, 2932, 97, 25, 74, 32, 218, 3584, 231, 387, 4, 1, 7093, 18, 1, 62, 149, 1664, 3, 788, 19, 258, 3712, 1109, 30, 1, 55, 11, 19, 1, 5120, 4, 608, 34, 8, 1, 4252, 1418, 90, 702, 2, 6108, 1627, 4, 3196, 39, 1268, 105, 2232, 85, 26, 162, 6, 176, 3, 9092, 281, 18, 30, 1, 127, 4, 1, 248, 1588, 5034, 2463, 24, 552, 50, 3383, 2, 44, 3, 608, 195, 128, 1109], [889, 1, 3475, 9457, 6, 28, 4, 145, 490, 36, 1, 498, 12, 183, 181, 2437, 10, 112, 194, 12, 24, 120, 13, 29, 12, 84, 11, 6, 24, 28, 2039, 3585, 4, 3, 819, 1407, 16, 52, 724, 1898, 45, 22, 190, 1, 516, 3210

In [11]:
word_index = tokenizer.word_index
print(f'{len(word_index)} unikatowych słów.')

88582 unikatowych słów.


In [12]:
# skracamy recenzje do pierwszych 100 słów
train_data = pad_sequences(sequences, maxlen=maxlen)
train_data.shape

(25000, 100)

In [13]:
train_labels = np.asarray(train_labels)
train_labels

array([0, 0, 0, ..., 1, 1, 1])

### Przemieszanie próbek

In [14]:
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

train_data.shape

(25000, 100)

### Podział na zbiór treningowy i walidacyjny

In [0]:
training_samples = 15000
validation_samples = 10000

X_train = train_data[:training_samples]
y_train = train_labels[:training_samples]
X_val = train_data[training_samples: training_samples + validation_samples]
y_val = train_labels[training_samples: training_samples + validation_samples]

### Budowa modelu

In [16]:
# Embedding(input_dim, output_dim)

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 16)                160016    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,160,033
Trainable params: 1,160,033
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [18]:
history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/5
15000/15000 [==============================] - 8s 506us/sample - loss: 0.4647 - accuracy: 0.7659 - val_loss: 0.3426 - val_accuracy: 0.8485
Epoch 2/5
15000/15000 [==============================] - 5s 350us/sample - loss: 0.1523 - accuracy: 0.9451 - val_loss: 0.4206 - val_accuracy: 0.8290
Epoch 3/5
15000/15000 [==============================] - 5s 339us/sample - loss: 0.0178 - accuracy: 0.9964 - val_loss: 0.5515 - val_accuracy: 0.8295
Epoch 4/5
15000/15000 [==============================] - 5s 347us/sample - loss: 0.0011 - accuracy: 0.9997 - val_loss: 0.7977 - val_accuracy: 0.8216
Epoch 5/5
15000/15000 [==============================] - 5s 339us/sample - loss: 2.5766e-04 - accuracy: 0.9999 - val_loss: 0.8982 - val_accuracy: 0.8285


In [19]:
def plot_hist(history):
  import pandas as pd
  import plotly.graph_objects as go

  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
  fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
  fig.update_layout(width=1000, height=500, title='Accuracy vs Val Accuracy', 
                    xaxis_title='Epoki', yaxis_title='Accuracy', yaxis_type='log')
  fig.show()

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
  fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
  fig.update_layout(width=1000, height=500, title='Loss vs Val Loss', 
                    xaxis_title='Epoki', yaxis_title='Loss', yaxis_type='log')
  fig.show()

plot_hist(history)

### Simple RNN
Model szybko zaczyna się przeuczać. Poprawiamy go stosując warstwy rekurencyjne.

In [0]:
from tensorflow.keras.layers import SimpleRNN, LSTM

In [22]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(units=16))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                784       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 320,801
Trainable params: 320,801
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/10
15000/15000 [==============================] - 42s 3ms/sample - loss: 0.5958 - accuracy: 0.6637 - val_loss: 0.4865 - val_accuracy: 0.7695
Epoch 2/10
15000/15000 [==============================] - 40s 3ms/sample - loss: 0.3611 - accuracy: 0.8497 - val_loss: 0.3784 - val_accuracy: 0.8395
Epoch 3/10
15000/15000 [==============================] - 40s 3ms/sample - loss: 0.2612 - accuracy: 0.8992 - val_loss: 0.3849 - val_accuracy: 0.8414
Epoch 4/10
15000/15000 [==============================] - 40s 3ms/sample - loss: 0.1857 - accuracy: 0.9312 - val_loss: 0.4149 - val_accuracy: 0.8302
Epoch 5/10
15000/15000 [==============================] - 42s 3ms/sample - loss: 0.1302 - accuracy: 0.9546 - val_loss: 0.4440 - val_accuracy: 0.8317
Epoch 6/10
15000/15000 [==============================] - 41s 3ms/sample - loss: 0.0815 - accuracy: 0.9729 - val_loss: 0.4890 - val_accuracy: 0.8287
Epoch 7/10
15000/15000 [==============================] 

In [25]:
plot_hist(history)

### Ocena modelu Simple RNN
Accuracy na poziomie testowym równe ok. 84%.

In [41]:
sequences = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(test_labels)

model.evaluate(X_test, y_test, verbose=0)

[0.4191080149912834, 0.8432]

### LSTM (long short-term memory)

In [26]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(units=16))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm (LSTM)                  (None, 16)                3136      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 323,153
Trainable params: 323,153
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [28]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/10
15000/15000 [==============================] - 12s 811us/sample - loss: 0.4628 - accuracy: 0.7771 - val_loss: 0.3452 - val_accuracy: 0.8546
Epoch 2/10
15000/15000 [==============================] - 6s 432us/sample - loss: 0.2902 - accuracy: 0.8809 - val_loss: 0.3286 - val_accuracy: 0.8581
Epoch 3/10
15000/15000 [==============================] - 6s 432us/sample - loss: 0.2396 - accuracy: 0.9042 - val_loss: 0.3956 - val_accuracy: 0.8416
Epoch 4/10
15000/15000 [==============================] - 6s 427us/sample - loss: 0.2040 - accuracy: 0.9207 - val_loss: 0.3650 - val_accuracy: 0.8501
Epoch 5/10
15000/15000 [==============================] - 6s 423us/sample - loss: 0.1790 - accuracy: 0.9317 - val_loss: 0.3566 - val_accuracy: 0.8511
Epoch 6/10
15000/15000 [==============================] - 6s 427us/sample - loss: 0.1606 - accuracy: 0.9383 - val_loss: 0.3678 - val_accuracy: 0.8503
Epoch 7/10
15000/15000 [=========================

In [29]:
plot_hist(history)

In [30]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(units=16))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_val, y_val))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 323,153
Trainable params: 323,153
Non-trainable params: 0
_________________________________________________________________
Train on 15000 samples, validate on 10000 samples
Epoch 1/3
15000/15000 [==============================] - 9s 567us/sample - loss: 0.5102 - accuracy: 0.7613 - val_loss: 0.3877 - val_accuracy: 0.8313
Epoch 2/3
15000/15000 [==============================] - 6s 427us/sample - loss: 0.2997 - accuracy: 0.8784 - val_loss: 0.3271 - val_accuracy: 0.8579
Epoch 3/3
15000/15000 [=======

### Ocena modelu LSTM
Accuracy na poziomie testowym równe ok. 85%.

In [33]:
model.evaluate(X_test, y_test, verbose=0)

[0.3404880493593216, 0.85328]

### GRU (Gated Recurrent Unit)

In [38]:
from tensorflow.keras.layers import GRU

model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(units=16))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
gru_1 (GRU)                  (None, 16)                2400      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 322,417
Trainable params: 322,417
Non-trainable params: 0
_________________________________________________________________
Train on 15000 samples, validate on 10000 samples
Epoch 1/10
15000/15000 [==============================] - 8s 526us/sample - loss: 0.5022 - accuracy: 0.7407 - val_loss: 0.3668 - val_accuracy: 0.8435
Epoch 2/10
15000/15000 [==============================] - 6s 426us/sample - loss: 0.3132 - accuracy: 0.8708 - val_loss: 0.3422 - val_accuracy: 0.8557
Epoch 3/10
15000/15000 [====

In [39]:
plot_hist(history)

### Ocena modelu GRU
Accuracy na poziomie testowym równe ok. 84%.

In [40]:
model.evaluate(X_test, y_test, verbose=0)

[0.4191080149912834, 0.8432]